In [160]:
import polars as pl
import polars.selectors as cs

from pathlib import Path

In [2]:
DATA_PATH = Path('..', 'data')
list(DATA_PATH.glob('*'))

[WindowsPath('../data/input_1.txt'),
 WindowsPath('../data/input_2.txt'),
 WindowsPath('../data/input_3.txt')]

In [5]:
df = (
    pl
    .read_csv(
        DATA_PATH / 'input_3.txt',
        has_header=False,
    )
)
print(df.shape)
df.head()

(140, 1)


column_1
str
"""..............…"
"""......429...83…"
""".........*...&…"
""".........900..…"
""".............7…"


In [144]:
df_row_col = (
    df
    .with_row_count('row')
    .with_columns(
        pl.col('column_1').str.split('')
    )
    .explode('column_1')
    .filter(
        pl.col('column_1') != ''
    )
    .with_columns(
        pl.lit(1).alias('ones'),
    )
    .select([
        pl.col('row'),
        pl.col('ones').cum_sum().sub(1).over('row').alias('column'),
        pl.col('column_1')
    ])
    .filter(
        pl.col('column_1') != '.'
    )
)
df_row_col_digit = (
    df_row_col
    .filter(
        pl.col('column_1').str.contains('\d')
    )
    .with_columns(
        pl
        .col('column')
        .sub(pl.col('column').shift(1))
        .ne(1)
        .cum_sum()
        .fill_null(0)
        .alias('is_group')
    )
)
df_row_col_symbol = (
    df_row_col
    .filter(
        ~ pl.col('column_1').str.contains('\d')
    )
)

In [179]:
display(
    df_row_col_digit
    .join(
        df_row_col_symbol,
        how='left',
        left_on=(
            pl.col('row'),
            pl.col('column'),
        ),
        right_on=(
            pl.col('row') + 1,
            pl.col('column'),
        ),
        suffix='_up'
    )
    .join(
        df_row_col_symbol,
        how='left',
        left_on=(
            pl.col('row'),
            pl.col('column'),
        ),
        right_on=(
            pl.col('row') - 1,
            pl.col('column'),
        ),
        suffix='_down'
    )
    .join(
        df_row_col_symbol,
        how='left',
        left_on=(
            pl.col('row'),
            pl.col('column'),
        ),
        right_on=(
            pl.col('row'),
            pl.col('column') + 1,
        ),
        suffix='_left'
    )
    .join(
        df_row_col_symbol,
        how='left',
        left_on=(
            pl.col('row'),
            pl.col('column'),
        ),
        right_on=(
            pl.col('row'),
            pl.col('column') - 1,
        ),
        suffix='_right'
    )
    .join(
        df_row_col_symbol,
        how='left',
        left_on=(
            pl.col('row'),
            pl.col('column'),
        ),
        right_on=(
            pl.col('row') + 1,
            pl.col('column') + 1,
        ),
        suffix='_up_left'
    )
    .join(
        df_row_col_symbol,
        how='left',
        left_on=(
            pl.col('row'),
            pl.col('column'),
        ),
        right_on=(
            pl.col('row') + 1,
            pl.col('column') - 1,
        ),
        suffix='_up_right'
    )
    .join(
        df_row_col_symbol,
        how='left',
        left_on=(
            pl.col('row'),
            pl.col('column'),
        ),
        right_on=(
            pl.col('row') - 1,
            pl.col('column') + 1,
        ),
        suffix='_down_left'
    )
    .join(
        df_row_col_symbol,
        how='left',
        left_on=(
            pl.col('row'),
            pl.col('column'),
        ),
        right_on=(
            pl.col('row') - 1,
            pl.col('column') - 1,
        ),
        suffix='_down_right'
    )
    .with_columns(
        (
            pl.col('column_1_up').is_not_null()
            | pl.col('column_1_down').is_not_null()
            | pl.col('column_1_left').is_not_null()
            | pl.col('column_1_right').is_not_null()
            | pl.col('column_1_up_left').is_not_null()
            | pl.col('column_1_up_right').is_not_null()
            | pl.col('column_1_down_left').is_not_null()
            | pl.col('column_1_down_right').is_not_null()
        )
        .alias('is_adjacent')
    )
    .filter(
        pl.col('is_adjacent').any().over('is_group')
    )
    .group_by('is_group')
    .agg(pl.col('column_1'))
    .select(
        pl.col('column_1').list.join('').cast(pl.Int32)
    )
    .sum()
)

column_1
i32
536202


# Part 2

In [206]:
df_row_col_digit_number = (
    df_row_col_digit
    .join(
        df_row_col_digit
        .group_by('is_group')
        .agg(pl.col('column_1'))
        .select(
            pl.col('is_group'),
            pl.col('column_1').list.join('').cast(pl.Int32).alias('number')
        ),
        how='inner',
        on=['is_group'],
    )
    .select('row', 'column', 'number')
)
df_row_col_digit_number

row,column,number
u32,i32,i32
0,44,411
0,45,411
0,46,411
0,68,363
0,69,363
0,70,363
0,73,134
0,74,134
0,75,134


In [221]:
with pl.Config(tbl_rows=1000):
    display(
        df_row_col_symbol
        .filter(
            pl.col('column_1') == '*'
        )
        .join(
            df_row_col_digit_number,
            how='left',
            left_on=(
                pl.col('row'),
                pl.col('column'),
            ),
            right_on=(
                pl.col('row') + 1,
                pl.col('column'),
            ),
            suffix='_up'
        )
        .join(
            df_row_col_digit_number,
            how='left',
            left_on=(
                pl.col('row'),
                pl.col('column'),
            ),
            right_on=(
                pl.col('row') - 1,
                pl.col('column'),
            ),
            suffix='_down'
        )
        .join(
            df_row_col_digit_number,
            how='left',
            left_on=(
                pl.col('row'),
                pl.col('column'),
            ),
            right_on=(
                pl.col('row'),
                pl.col('column') + 1,
            ),
            suffix='_left'
        )
        .join(
            df_row_col_digit_number,
            how='left',
            left_on=(
                pl.col('row'),
                pl.col('column'),
            ),
            right_on=(
                pl.col('row'),
                pl.col('column') - 1,
            ),
            suffix='_right'
        )
        .join(
            df_row_col_digit_number,
            how='left',
            left_on=(
                pl.col('row'),
                pl.col('column'),
            ),
            right_on=(
                pl.col('row') + 1,
                pl.col('column') + 1,
            ),
            suffix='_up_left'
        )
        .join(
            df_row_col_digit_number,
            how='left',
            left_on=(
                pl.col('row'),
                pl.col('column'),
            ),
            right_on=(
                pl.col('row') + 1,
                pl.col('column') - 1,
            ),
            suffix='_up_right'
        )
        .join(
            df_row_col_digit_number,
            how='left',
            left_on=(
                pl.col('row'),
                pl.col('column'),
            ),
            right_on=(
                pl.col('row') - 1,
                pl.col('column') + 1,
            ),
            suffix='_down_left'
        )
        .join(
            df_row_col_digit_number,
            how='left',
            left_on=(
                pl.col('row'),
                pl.col('column'),
            ),
            right_on=(
                pl.col('row') - 1,
                pl.col('column') - 1,
            ),
            suffix='_down_right'
        )
        .rename({
            'number': 'number_up'
        })
        .with_columns(
            pl
            .concat_list(
                cs.contains('number')
            )
            .list
            .eval(
                pl.element().drop_nulls().unique()
            )
            .alias('numbers')
        )
        .filter(
            pl.col('numbers').list.len() == 2
        )
        .with_columns(
            pl.col('numbers').list[0] * pl.col('numbers').list[1]
        )
        .sum()
    )

row,column,column_1,number_up,number_down,number_left,number_right,number_up_left,number_up_right,number_down_left,number_down_right,numbers
u32,i32,str,i32,i32,i32,i32,i32,i32,i32,i32,i32
22237,21750,null,64067,69003,23246,22831,77684,88148,75875,77129,78272573
